In [65]:
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from loguru import logger
from scipy.optimize import minimize
from netCDF4 import Dataset

In [2]:
base_path="/home/tzhang/wrf_solar/wsolar412_bnl/runwrf_tune"
archive_path = "/S2/gscr2/tzhang/big_data/UQ/tune/"
para_names = ['vdis','beta_con']
os.chdir(base_path)

In [6]:
def delete_all_case():
    os.system("rm -rf "+archive_path+"/Iter*")

# check bounds

In [58]:
para_bounds = {
    'vdis':[0.01,1.4],
    'beta_con':[1.02E20, 1.67e24]
}

def check_bounds(x):
    nx = len(x)
    nb = len(para_bounds)
    
    if nx != nb:
        raise SystemExit('shape of input parameters should be consistent with the bounds!')
    
    for key in x:
        x[key] = x[key] if x[key] > para_bounds[key][0] else para_bounds[key][0]
        x[key] = x[key] if x[key] < para_bounds[key][1] else para_bounds[key][1]

    return x

x = {'vdis':1.5, 'beta_con':0.2}
aa = check_bounds(x)

print(aa)

{'vdis': 1.4, 'beta_con': 1.02e+20}


# calc metrics

In [72]:
def calc_metrics(path_mod):
    var_list = ['dni','dhi'] # dni: direct normal irradiance DHI: diffuse horizontal irradiance
    varn_obs = ['obs_swddni', 'obs_swddif']
    varn_mod = ['SWDDNI','SWDDIF']
    
    path_base = "/ss/hsdc/home/tzhang/wrf_solar/"
    path_obs = path_base+"sgpradflux10long_area_mean.c1.20160619_1200UTC.nc"
    path_def = "/S2/gscr2/tzhang/big_data/UQ/tune/runwrf_def/wrfout_d02_2016-06-19_06:00:00"
    
    fid_obs = Dataset(path_obs)
    fid_def = Dataset(path_def)
    fid_mod = Dataset(path_mod)
    
    Chi = 0
    for i, var in enumerate(var_list):
        #print(var)
        
        var_obs = fid_obs.variables[varn_obs[i]][:]
        var_mod = fid_mod.variables[varn_mod[i]][36:]
        var_def = fid_def.variables[varn_mod[i]][36:]
        
        var_mod_avg = np.mean(var_mod, axis=(1,2))
        var_def_avg = np.mean(var_def, axis=(1,2))
        
        #print(var_obs.shape)
        #print(var_mod_avg.shape)
        #print(var_def_avg.shape)
        
        theta_mod = 0
        for j in range(var_obs.shape[0]):
            theta_mod += (var_obs[j] - var_mod_avg[j]) ** 2
            
        theta_def = 0
        for j in range(var_obs.shape[0]):
            theta_def += (var_obs[j] - var_def_avg[j]) ** 2
            
        Chi += (theta_mod / theta_def)
        
    Chi /= len(var_list)
    
    return Chi
            
        
        
#calc_metrics("/S2/gscr2/tzhang/big_data/UQ/sampling/case0/wrfout_d02_2016-06-19_06:00:00")
calc_metrics("/S2/gscr2/tzhang/big_data/UQ/tune/runwrf_def/wrfout_d02_2016-06-19_06:00:00")

1.0

# run case

In [83]:
ite = -1

def run_case(x):
    global ite
    ite = ite + 1 
    
    # check parameter bound
    x = check_bounds(x)
    
    # modify namelist
    logger.info("Iter"+str(ite)+": modifiy namelist")
    for key in x:
        replace_str = "sed -i '/^ "+key+"/c\ "+key+"="+str(x[key])+"' namelist.input"
        os.system(replace_str)

    # run model
    logger.info("Iter"+str(ite)+": run model")
    time.sleep(30)
    #jid = os.popen("qsub runwrf.sh").read().split('.')[0]
    #print(jid)
    #while os.popen("qstat").read().find(jid) != -1:
    #    time.sleep(300)

    #archive case
    logger.info("Iter"+str(ite)+": archive case")
    os.system("mkdir -p "+archive_path+"/Iter"+str(ite))
    os.system("cp namelist.input "+archive_path+"/Iter"+str(ite))
    os.system("cp wrfout_* "+archive_path+"/Iter"+str(ite))
    
    logger.info("case"+str(ite)+": success!!!")
    #logger.remove()
    
def run_case_wrapper(x):
    paras = {}
    
    for i,key in enumerate(para_names):
        paras[key] = x[i]
        
    mcpi = run_case(paras)
    return mcpi

2021-02-20 02:29:46.924 | INFO     | __main__:run_case:20 - Iter0: modifiy namelist
2021-02-20 02:29:47.014 | INFO     | __main__:run_case:26 - Iter0: run model
2021-02-20 02:30:17.048 | INFO     | __main__:run_case:34 - Iter0: archive case
2021-02-20 02:30:43.684 | INFO     | __main__:run_case:39 - case0: success!!!


In [80]:
x0 = np.array([1.311735, 1.1281180989999998e+24])

initial_simplex = np.array([
    [1.311735, 1.1281180989999998e+24],
    [1.3103449999999999, 4.2008134699999996e+23],
    [0.869715, 1.4955787099999998e+23]
])

res = minimize(run_case_wrapper, x0, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True, 'return_all':True})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 150
         Function evaluations: 270


In [86]:
def rosen(x):
    """The Rosenbrock function"""
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

x0 = np.array([1.3, 0.7, 0.8])

initial_simplex = np.array([
    [1.1, 1.7, 2.8],
    [0.1, 1.17, 1.8],
    [1.01, 2.7, 4.8],
    [4.01, 4.7, 3.8]
])

res = minimize(rosen, x0, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True, 'return_all':True, 'initial_simplex':initial_simplex})

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 117
         Function evaluations: 217


In [87]:
print(res)

       allvecs: [array([1.1, 1.7, 2.8]), array([1.1, 1.7, 2.8]), array([1.1, 1.7, 2.8]), array([1.1, 1.7, 2.8]), array([0.86666667, 0.39703704, 0.39259259]), array([0.86666667, 0.39703704, 0.39259259]), array([0.92712963, 0.97184671, 0.81954733]), array([0.92712963, 0.97184671, 0.81954733]), array([0.92712963, 0.97184671, 0.81954733]), array([0.92712963, 0.97184671, 0.81954733]), array([1.00378472, 1.1406848 , 1.35054012]), array([1.00378472, 1.1406848 , 1.35054012]), array([0.90942033, 0.82905312, 0.79203747]), array([0.92158452, 0.94895661, 0.89419117]), array([1.05494652, 1.13733474, 1.26058504]), array([1.05494652, 1.13733474, 1.26058504]), array([1.05494652, 1.13733474, 1.26058504]), array([1.05494652, 1.13733474, 1.26058504]), array([0.95564111, 0.92926116, 0.89157888]), array([0.95289434, 0.90373668, 0.81610189]), array([0.95289434, 0.90373668, 0.81610189]), array([0.95289434, 0.90373668, 0.81610189]), array([0.95289434, 0.90373668, 0.81610189]), array([0.95289434, 0.90373668, 0